#Natural Language Processing 
Natural Language Processing (or NLP for short) is a discipline in computing that deals with the communication between natural (human) languages and computer languages. A common example of NLP is something like spellcheck or autocomplete. Essentially NLP is the field that focuses on how computers can understand and/or process natural/human languages. 

###Recurrent Neural Networks

In this tutorial we will introduce a new kind of neural network that is much more capable of processing sequential data such as text or characters called a **recurrent neural network** (RNN for short). 

We will learn how to use a reccurent neural network to do the following:
- Sentiment Analysis
- Character Generation 

RNN's are complex and come in many different forms so in this tutorial we wil focus on how they work and the kind of problems they are best suited for.



## Sequence Data
In the previous tutorials we focused on data that we could represent as one static data point where the notion of time or step was irrelevant. Take for example our image data, it was simply a tensor of shape (width, height, channels). That data doesn't change or care about the notion of time. 

In this tutorial we will look at sequences of text and learn how we can encode them in a meaningful way. Unlike images, sequence data such as long chains of text, weather patterns, videos and really anything where the notion of a step or time is relevant needs to be processed and handled in a special way. 

But what do I mean by sequences and why is text data a sequence? Well that's a good question. Since textual data contains many words that follow in a very specific and meaningful order, we need to be able to keep track of each word and when it occurs in the data. Simply encoding say an entire paragraph of text into one data point wouldn't give us a very meaningful picture of the data and would be very difficult to do anything with. This is why we treat text as a sequence and process one word at a time. We will keep track of where each of these words appear and use that information to try to understand the meaning of peices of text.



##Encoding Text
As we know machine learning models and neural networks don't take raw text data as an input. This means we must somehow encode our textual data to numeric values that our models can understand. There are many different ways of doing this and we will look at a few examples below. 

Before we get into the different encoding/preprocessing methods let's understand the information we can get from textual data by looking at the following two movie reviews.

```I thought the movie was going to be bad, but it was actually amazing!```

```I thought the movie was going to be amazing, but it was actually bad!```

Although these two setences are very similar we know that they have very different meanings. This is because of the **ordering** of words, a very important property of textual data.

Now keep that in mind while we consider some different ways of encoding our textual data.

###Bag of Words
The first and simplest way to encode our data is to use something called **bag of words**. This is a pretty easy technique where each word in a sentence is encoded with an integer and thrown into a collection that does not maintain the order of the words but does keep track of the frequency. Have a look at the python function below that encodes a string of text into bag of words. 

In [ ]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


This isn't really the way we would do this in practice, but I hope it gives you an idea of how bag of words works. Notice that we've lost the order in which words appear. In fact, let's look at how this encoding works for the two sentences we showed above.



In [ ]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


We can see that even though these sentences have a very different meaning they are encoded exaclty the same way. Obviously, this isn't going to fly. Let's look at some other methods.



###Integer Encoding
The next technique we will look at is called **integer encoding**. This involves representing each word or character in a sentence as a unique integer and maintaining the order of these words. This should hopefully fix the problem we saw before were we lost the order of words.


In [ ]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


And now let's have a look at one hot encoding on our movie reviews.

In [ ]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


Much better, now we are keeping track of the order of words and we can tell where each occurs. But this still has a few issues with it. Ideally when we encode words, we would like similar words to have similar labels and different words to have very different labels. For example, the words happy and joyful should probably have very similar labels so we can determine that they are similar. While words like horrible and amazing should probably have very different labels. The method we looked at above won't be able to do something like this for us. This could mean that the model will have a very difficult time determing if two words are similar or not which could result in some pretty drastic performace impacts.



###Word Embeddings
Luckily there is a third method that is far superior, **word embeddings**. This method keeps the order of words intact as well as encodes similar words with very similar labels. It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.

Unlike the previous techniques word embeddings are learned by looking at many different training examples. You can add what's called an *embedding layer* to the beggining of your model and while your model trains your embedding layer will learn the correct embeddings for words. You can also use pretrained embedding layers.

This is the technique we will use for our examples and its implementation will be showed later on.



##Recurrent Neural Networks (RNN's)
Now that we've learned a little bit about how we can encode text it's time to dive into recurrent neural networks. Up until this point we have been using something called **feed-forward** neural networks. This simply means that all our data is fed forwards (all at once) from left to right through the network. This was fine for the problems we considered before but won't work very well for processing text. After all, even we (humans) don't process text all at once. We read word by word from left to right and keep track of the current meaning of the sentence so we can understand the meaning of the next word. Well this is exaclty what a recurrent neural network is designed to do. When we say recurrent neural network all we really mean is a network that contains a loop. A RNN will process one word at a time while maintaining an internal memory of what it's already seen. This will allow it to treat words differently based on their order in a sentence and to slowly build an understanding of the entire input, one word at a time.

This is why we are treating our text data as a sequence! So that we can pass one word at a time to the RNN.

Let's have a look at what a recurrent layer might look like.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Source: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*

Let's define what all these variables stand for before we get into the explination.

**h<sub>t</sub>** output at time t

**x<sub>t</sub>** input at time t

**A** Recurrent Layer (loop)

What this diagram is trying to illustrate is that a recurrent layer processes words or input one at a time in a combination with the output from the previous iteration. So, as we progress further in the input sequence, we build a more complex understanding of the text as a whole.

What we've just looked at is called a **simple RNN layer**. It can be effective at processing shorter sequences of text for simple problems but has many downfalls associated with it. One of them being the fact that as text sequences get longer it gets increasingly difficult for the network to understand the text properly.



##LSTM
The layer we dicussed in depth above was called a *simpleRNN*. However, there does exist some other recurrent layers (layers that contain a loop) that work much better than a simple RNN layer. The one we will talk about here is called LSTM (Long Short-Term Memory). This layer works very similarily to the simpleRNN layer but adds a way to access inputs from any timestep in the past. Whereas in our simple RNN layer input from previous timestamps gradually disappeared as we got further through the input. With a LSTM we have a long-term memory data structure storing all the previously seen inputs as well as when we saw them. This allows for us to access any previous value we want at any point in time. This adds to the complexity of our network and allows it to discover more useful relationships between inputs and when they appear. 

For the purpose of this course we will refrain from going any further into the math or details behind how these layers work.



##Sentiment Analysis
And now time to see a recurrent neural network in action. For this example, we are going to do something called sentiment analysis.

The formal definition of this term from Wikipedia is as follows:

*the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.*

The example we’ll use here is classifying movie reviews as either postive, negative or neutral.

*This guide is based on the following tensorflow tutorial: https://www.tensorflow.org/tutorials/text/text_classification_rnn*



###Movie Review Dataset
Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.
 




In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
# Lets look at one review
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

###More Preprocessing
If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:
- if the review is greater than 250 words then trim off the extra words
- if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

Luckily for us keras has a function that can do this for us:




In [ ]:
train_data = sequence.pad_sequences(train_data, MAXLEN) # pads_sequences adjusts the length to the argument of MAXLEN (250)
test_data = sequence.pad_sequences(test_data, MAXLEN)

###Creating the Model
Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment. 

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32), # Embedding layer requires data be integer encoded (integer and order). Creates 32 dim. vector for every word
    tf.keras.layers.LSTM(32), # LSTM layer with a 32 dimensionality output space.
    tf.keras.layers.Dense(1, activation="sigmoid") # Classification layer with one single neuron and sigmoid activation (only 2 classes)
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


###Training
Now it's time to compile and train the model. 

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc']) #binary crossentropy cause we only have two classes

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 61s 98ms/step - loss: 0.4192 - acc: 0.8092 - val_loss: 0.2841 - val_acc: 0.8830
Epoch 2/10
625/625 [==============================] - 60s 96ms/step - loss: 0.2388 - acc: 0.9087 - val_loss: 0.2678 - val_acc: 0.8866
Epoch 3/10
625/625 [==============================] - 60s 96ms/step - loss: 0.1849 - acc: 0.9325 - val_loss: 0.2664 - val_acc: 0.8912
Epoch 4/10
625/625 [==============================] - 60s 95ms/step - loss: 0.1494 - acc: 0.9478 - val_loss: 0.2832 - val_acc: 0.8800
Epoch 5/10
625/625 [==============================] - 60s 95ms/step - loss: 0.1290 - acc: 0.9550 - val_loss: 0.3558 - val_acc: 0.8888
Epoch 6/10
625/625 [==============================] - 60s 96ms/step - loss: 0.1091 - acc: 0.9614 - val_loss: 0.3347 - val_acc: 0.8770
Epoch 7/10
625/625 [==============================] - 60s 96ms/step - loss: 0.1010 - acc: 0.9663 - val_loss: 0.3276 - val_acc: 0.8864
Epoch 8/10
625/625 [==============================] - 60s 96ms

And we'll evaluate the model on our training data to see how well it performs.

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 17s 21ms/step - loss: 0.4316 - acc: 0.8548
[0.43164724111557007, 0.8547999858856201]


So we're scoring somewhere in the mid-high 80's. Not bad for a simple recurrent network.

###Making Predictions
Now let’s use our network to make predictions on our own reviews. 

Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.




In [ ]:
# get every index dictionary of the imdb dataset.
word_index = imdb.get_word_index()

# pre-process some text like our training data
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text) # convert all the text into tokens (individual words themself)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens] # if the individual word is in IMDB dictionary then write the integer, else 0
  return sequence.pad_sequences([tokens], MAXLEN)[0] # adjusts the lenght of the review to 250 (MAXLEN)

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [ ]:
# while were at it lets make a decode function to translate back again to words

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD: # if the number is not zero
        text += reverse_word_index[num] + " " # add the integer/word to the text variable plus a space

    return text[:-1] # return everything except the last space added
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
''' now time to make a prediction 
 - (0 - 0.5) = negative review. 
 - (0.5 - 1) = positive review)'''

def predict(text): # argument text is the movie review
  encoded_text = encode_text(text) # use the function above to preprocess the text and get the integers
  pred = np.zeros((1,250)) #creates np array 1x250 (1 row, 250 columns) of zeros. (250 is the length of the review)
  pred[0] = encoded_text # insert the review with integers into the array we created
  result = model.predict(pred) # predict on that array
  print(result[0]) # print the first result of the predict

positive_review = "That movie was awesome! I really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "That movie really sucked. I hated it and wouldn't watch it again. It was one of the worst things I've ever watched"
predict(negative_review)


[0.8895886]
[0.35574287]


##RNN Play Generator

Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.


*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


###Dataset
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.




In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


###Loading Your Own Data
To load your own data, you'll need to upload a file from the dialog below. Then you'll need to follow the steps from above but load in this new file instead.



In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

###Read Contents of File
Let's look at the contents of the file.

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



###Encoding
Since this text isn't encoded yet well need to do that ourselves. We are going to encode each unique character as a different integer.



In [ ]:
vocab = sorted(set(text)) # sort all the unique characters in the text
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)} # gives a number for every single letter in the vocabulary (letter to index)
idx2char = np.array(vocab) # turn the initial vocabulary into a list or array

def text_to_int(text): # takes some text and converts it to an integer
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text) #converts the entire file into integers

In [ ]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


And here we will make a function that can convert our numeric values to text.


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy() # converts integers into a numpy array first
  except:
    pass # doesn't need to convert into a numpy array cause it's already one
  return ''.join(idx2char[ints]) # join all the characters from idx2char[ints] into the ''

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1) # examples per epoch = number of characters of the file / 101 (100 + 1 because of the output sequence)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) # converts the integers of the text to a stream of integers, slicing the tensor of integers

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) # turn the stream into batches of size 101 and drop the rest if it's the case

Now we need to use these sequences of length 101 and split them into input and output.

In [ ]:
# all is in integers right now but we put the example in characters for better understanding
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
'''Example of what we have just done:
We takes the two first batches of our sequences and prints their input and output sequences'''
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x)) # converts the integers into the input text sequence
  print("\nOUTPUT")
  print(int_to_text(y)) # converts the integers into the output text sequence



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches.

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters (65)
EMBEDDING_DIM = 256 # represents how big are the vectors that represents the words
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True) # we shuffle all the dataset (sequences) with batches of 64 sequences

###Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, # input=vocab_size, output=embedding_dim, and shape of the batch
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, # number of nodes
                        return_sequences=True, # for seeing the full sequence in the output sequence, and not only the last output
                        stateful=True, # If True, the last state for each sample at index i in a batch will be used as initial state in the following batch
                        recurrent_initializer='glorot_uniform'), # good default to pick
    tf.keras.layers.Dense(vocab_size) # final layer has the amount of characters in the vocabulary
  ])
  return model #returns all the model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE) # (65,256,1024,64)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


###Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch and because TensorFlow doesn't have a built-in loss function that can look at a 3D nested array of probabilities.



However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.



In [ ]:
for input_example_batch, target_example_batch in data.take(1): # this gives us the output of the final layer from the first example of the dataset
  example_batch_predictions = model(input_example_batch)  # ask our model before it's trained for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the prediction is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions)) # print the length of the first batch of our training data
print(example_batch_predictions) # print the prediction of the first batch of our training data. We got a 3D array with the predictions of every timestep of
# every training example (sequence) in the batch

64
tf.Tensor(
[[[ 3.59557755e-03  4.56055568e-04 -8.62357672e-03 ...  4.83387522e-03
    3.42590362e-03 -6.12006476e-03]
  [ 5.53392433e-03 -7.16851233e-03 -7.03548081e-04 ...  2.88809300e-03
    1.28260267e-03 -6.48903009e-03]
  [ 7.07764877e-03 -1.23533979e-02  5.27610630e-03 ...  2.31968518e-03
   -2.17556953e-04 -7.45583465e-03]
  ...
  [-8.59005377e-06 -6.07613940e-03  2.53517320e-03 ... -6.88940403e-04
   -4.97948844e-03 -3.48728057e-03]
  [ 2.65167095e-04 -6.59023505e-03 -4.43567149e-03 ...  3.11200740e-03
   -3.24592763e-03 -6.24498818e-03]
  [ 5.21713495e-03 -4.04186221e-03 -8.11704062e-03 ...  6.54191338e-03
   -3.94967292e-03 -4.94064530e-04]]

 [[-2.54827528e-03 -5.29280957e-03  2.43321015e-03 ... -7.22216500e-04
    5.70662785e-03  2.18828442e-03]
  [-3.05462745e-03  1.17260357e-03 -1.88945141e-03 ...  3.98078980e-03
   -1.53880566e-04  3.22312280e-03]
  [-5.06061036e-03 -5.00811916e-03 -7.03881774e-03 ...  6.41302159e-03
    5.73457964e-03  3.42984567e-03]
  ...
  [-1.786

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred)) # print the length of the very first training example inside the first batch
print(pred) # print the prediction: for every single training example inside the first batch we get *length* (100 in this case) outputs
# notice this is a 2d array of length 100 (100 time steps), where each interior array is the prediction for the next character at EACH time step, so we got
# a 2D array with the predictions of every timestep in the first training example (sequence) of the batch.

100
tf.Tensor(
[[ 3.5955776e-03  4.5605557e-04 -8.6235767e-03 ...  4.8338752e-03
   3.4259036e-03 -6.1200648e-03]
 [ 5.5339243e-03 -7.1685123e-03 -7.0354808e-04 ...  2.8880930e-03
   1.2826027e-03 -6.4890301e-03]
 [ 7.0776488e-03 -1.2353398e-02  5.2761063e-03 ...  2.3196852e-03
  -2.1755695e-04 -7.4558347e-03]
 ...
 [-8.5900538e-06 -6.0761394e-03  2.5351732e-03 ... -6.8894040e-04
  -4.9794884e-03 -3.4872806e-03]
 [ 2.6516709e-04 -6.5902350e-03 -4.4356715e-03 ...  3.1120074e-03
  -3.2459276e-03 -6.2449882e-03]
 [ 5.2171350e-03 -4.0418622e-03 -8.1170406e-03 ...  6.5419134e-03
  -3.9496729e-03 -4.9406453e-04]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred)) # print the length of the very first timestep inside the first training example inside the first batch
print(time_pred) # print the prediction: for every single timestep inside 1st training example inside the first batch we get *length* (65 in this case) outputs
# and of course its 65 values representing the probability of each character occuring next

65
tf.Tensor(
[ 0.00359558  0.00045606 -0.00862358  0.00399215 -0.00171073 -0.0016705
  0.00180821  0.00209641 -0.00526769 -0.00056423  0.00077079  0.00373395
  0.00170076 -0.0033618   0.00232039 -0.0049313  -0.00088408 -0.00131902
  0.00604895  0.00197199 -0.00322425  0.00106036  0.00048937  0.00287598
  0.00300145  0.00214393 -0.00133249  0.00264432 -0.00076504  0.00269905
  0.0007117  -0.00248711 -0.00078955 -0.00370027  0.00261119 -0.00417354
  0.00139081  0.00194199 -0.00490035  0.00480163  0.00215964  0.00332507
 -0.00130802 -0.00226522  0.00170921  0.00339936  0.00032932 -0.00126746
  0.00395518  0.00303233  0.00180264  0.00173981  0.00087665  0.00601361
 -0.00025024  0.00152826  0.00279757 -0.00085418 -0.00718535 -0.00511202
  0.00205498  0.00068444  0.00483388  0.0034259  -0.00612006], shape=(65,), dtype=float32)


In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probability distribution)
sampled_indices = tf.random.categorical(pred, num_samples=1) # sample the 2D array of the prediction of every timestep of the first training example in batch

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0] # change the shape of the sample (100,65) to ¿(1,100)?
predicted_chars = int_to_text(sampled_indices) # converts the numbers to the actual characters

predicted_chars  # show what the model predicted for every timestep in training sequence 1: all the predicted characters that it had

'tPVHUxaItYOW,utsF!t.DWeZjWqsZze-3uljoY3qSr&qgwvHeMG.zIlvA CpJmbhZD3RfSu:lHPupF;YQ$IqDIwFG:achPR-iZAa'

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were. 

In [ ]:
# this loss function takes all the labels and all the probability distributions and compute the loss to see how different or similar these two things are
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True) # Use this loss function when there are two or more labels

###Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next. 


In [ ]:
model.compile(optimizer='adam', loss=loss) # compile the model with 'adam' optimizer and the loss defined above

###Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # ./training_checkpoints/ckpt_{epoch}

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, # path to save the model file
    save_weights_only=True) #  only the model's weights will be saved

###Training
Finally, we will start training the model. 

**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**



In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 9s 54ms/step - loss: 2.7606
Epoch 2/50
172/172 [==============================] - 9s 54ms/step - loss: 2.0729
Epoch 3/50
172/172 [==============================] - 9s 54ms/step - loss: 1.8164
Epoch 4/50
172/172 [==============================] - 9s 54ms/step - loss: 1.6635
Epoch 5/50
172/172 [==============================] - 9s 54ms/step - loss: 1.5661
Epoch 6/50
172/172 [==============================] - 9s 54ms/step - loss: 1.4982
Epoch 7/50
172/172 [==============================] - 9s 54ms/step - loss: 1.4480
Epoch 8/50
172/172 [==============================] - 9s 54ms/step - loss: 1.4102
Epoch 9/50
172/172 [==============================] - 9s 54ms/step - loss: 1.3783
Epoch 10/50
172/172 [==============================] - 9s 54ms/step - loss: 1.3519
Epoch 11/50
172/172 [==============================] - 9s 54ms/step - loss: 1.3281
Epoch 12/50
172/172 [==============================] - 9s 54ms/step - loss: 1.3066
Epoch 13/50
1

###Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1) # batch_size of 1, so we can pass some sequence of any length

Once the model is finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.



In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # load weights of the latest checkpoint
model.build(tf.TensorShape([1, None])) #build the model manually: input [1 (for the batch I think), None (don't know what the next dimension length will be)]

We can load **any checkpoint** we want by specifying the exact file to load.

In [ ]:
checkpoint_num = 10 # this is for example for loading the checkpoint of epoch 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

###Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [ ]:
# Evaluation step (generating text using the learned model)
def generate_text(model, start_string): # function from TensorFlow website
  num_generate = 800  # Number of characters to generate

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string] # string to number for every character in the argument start_string
  input_eval = tf.expand_dims(input_eval, 0) # add one more dimension to input_eval 

  text_generated = [] # Empty string to store our results

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states() # reset_states to clear the last state that model remembered
  for i in range(num_generate): # For 0 to 800...
      predictions = model(input_eval) # prediction of input_eval
      predictions = tf.squeeze(predictions, 0) # remove the batch dimension (the extra dimension)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0) # add predicted_id to input_eval adding one more dimension

      text_generated.append(idx2char[predicted_id]) # append to the empty list the predicted character converted back from integers to letters

  return (start_string + ''.join(text_generated)) # returns the initial string plus all the characters stored in the list text_generated

In [ ]:
inp = input("Type a starting string: ") # you type here a string that will activate the sequence
print(generate_text(model, inp)) #print the text using the function defined above

Type a starting string: you
you?

TRANIE:
Give me this triumph of my kind lord's stears,
Which kings come, woe:
'Tis very good son.

PAULINA:
Pardon me not!
O woful deare!
Prepare her strength, my loving, poor his,
And he hath temption to be moved him.

GLOUCESTER:
Your good's queen, to have a rush'd before.
My wife is left heard greatness as Edward,
Your person prospers her revengefully to a tackled slave,
Unless the duke as lift before the peace
To this and unpil disgrace to the ur sit and goes to love,
Sufficially in question,
But far object to these griefs,
Which often
As eyes daughters, while he heard it toward the town-day
How I grim at thy father's maids
Of helpation as you can leave his resolver:
Life and his wives with gold, whose purpose
Of his thampless.

LEONTES:
Go, Some Paris is eence, God we proceed I bou


*And* that's pretty much it for this module! I highly reccomend messing with the model we just created and seeing what you can get it to do!

##Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Classification with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_classification_rnn.
3. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
4. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.